In [117]:
from langchain_community.utilities import SQLDatabase

In [118]:
db = SQLDatabase.from_uri("sqlite:///../data/movie.db")

In [119]:
print(db.get_usable_table_names())

['IMDB', 'earning', 'genre']


In [120]:
db_schema = db.get_table_info()

In [121]:
from dotenv import load_dotenv
import os
import google.generativeai as genai

load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=google_api_key)

In [122]:
model = genai.GenerativeModel('gemini-pro')

In [123]:
model

genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
)

In [124]:
question = "How many titles do we have in the database?"

In [125]:
response = model.generate_content(question)

In [126]:
response.text

'I do not have access to your database, so I cannot answer this question. You can find the number of titles in your database by querying the database.'

In [127]:
few_shots = [
        {
            "input": "How many Titles are present?", 
            "query": "SELECT COUNT(*) FROM IMDB;"},
        {
            "input": "List the Titles with the 10 highest Rating",
            "query": "SELECT Title FROM IMDB ORDER BY Rating DESC LIMIT 10;",
        },
        {
            "input": "List the genre of the highest worldwide earning title",
            "query": "SELECT a.GENRE FROM earning b LEFT JOIN genre a ON a.Movie_id = b.Movie_id ORDER BY b.Worldwide DESC LIMIT 1;",
        },
        {
            "input": "List the Title, Budget and domestic earnings from the highest Rating Title",
            "query": "SELECT a.Title, a.Budget, b.Domestic FROM IMDB a LEFT JOIN earnings b ON a.Movie_id = b.Movie_id ORDER BY a.Ratings DESC LIMIT 1;",
        }
    ]

In [128]:
prompt = [
        f"""
        You are an expert in converting English questions to SQL query!
        The SQL database has 3 tables, and these are the schemas: {db_schema}. 
        You can order the results by a relevant column to return the most interesting examples in the database.
        Never query for all the columns from a specific table, only ask for the relevant columns given the question.
        Also the sql code should not have ``` in beginning or end and sql word in output.
        You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
    
        DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

        If the question does not seem related to the database, just return "I don't know" as the answer.

        Here are some examples of user inputs and their corresponding SQL queries:
        """,
    ]

# Append each example to the prompt
for sql_example in few_shots:
    prompt.append(f"\nExample - {sql_example['input']}, the SQL command will be something like this {sql_example['query']}")

# Join prompt sections into a single string
formatted_prompt = [''.join(prompt)]

In [129]:
response_with_few_shots = model.generate_content([formatted_prompt[0], question])

In [130]:
response_with_few_shots

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': 'SELECT COUNT(*) FROM IMDB;'}], 'role': 'model'}, 'finish_reason': 1, 'index': 0, 'safety_ratings': [{'category': 9, 'probability': 1, 'blocked': False}, {'category': 8, 'probability': 1, 'blocked': False}, {'category': 7, 'probability': 1, 'blocked': False}, {'category': 10, 'probability': 1, 'blocked': False}], 'token_count': 0, 'grounding_attributions': []}]}),
)

In [131]:
question_related_to_avg = "What's the average rating of the Titles?"

In [132]:
response_related_to_avg = model.generate_content([formatted_prompt[0], question_related_to_avg])
response_related_to_avg.text

'SELECT \nAVG(Rating) \nFROM \nIMDB'

In [133]:
response_related_to_avg.text

'SELECT \nAVG(Rating) \nFROM \nIMDB'

In [134]:
sql_response = db.run('SELECT AVG(Rating) FROM IMDB')

In [135]:
from langchain_google_vertexai import VertexAI

credentials_path = "credentials.json"

project_id = "sql-llm-project-427811"

llm = VertexAI(model_name='gemini-pro', project=project_id, credentials_path=credentials_path)


In [146]:
llm.invoke(f"""
Based on the sql response, write an intuitive answer:

SQL response: {sql_response}
""")

'The average response time from our server based on the last 7 days of data is 7.87 seconds. '

In [138]:
from langchain_core.prompts import PromptTemplate

In [139]:
template = """
Based on the sql response, write an intuitive answer:

SQL response: {sql_response}
"""

In [140]:
prompt_template = PromptTemplate(template = template, input_variables = ['sql_response'])

In [141]:
from langchain.chains import LLMChain

In [142]:
chain = LLMChain(llm = llm, prompt = prompt_template)

In [145]:
chain.invoke(sql_response)

{'sql_response': '[(7.873504273504273,)]',
 'text': 'The average price of the product is $7.87.'}